In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb
from time import time
import gc
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [3]:
#path = '/Users/804357/Desktop/MyFiles/Learn/LOM/Data/'
path = '/Users/Vishy/Files/AVDatahack/LOM/Data/' 
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
camp = pd.read_csv(path+'campaign_data.csv')

train_df = pd.merge(train, camp, on='campaign_id')
test_df = pd.merge(test, camp, on='campaign_id')
print('Shape of the train data is', train_df.shape)
print('Shape of the test data is', test_df.shape)

"""len_train = len(train_df)
train_df = train_df.append(test_df)
del test_df
gc.collect()"""

Shape of the train data is (1023191, 14)
Shape of the test data is (773858, 12)


'len_train = len(train_df)\ntrain_df = train_df.append(test_df)\ndel test_df\ngc.collect()'

In [3]:
print('New variable creations')
train_df['hour'] = pd.to_datetime(train_df.send_date).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.send_date).dt.day.astype('uint8')
train_df['dow'] = pd.to_datetime(train_df.send_date).dt.dayofweek.astype('uint8')

New variable creations


In [4]:
train_df['dow'] = train_df['dow'].map({0:'Mon', 1:'Tue', 2:'Wed', 3:'Thr',4: 'Fri', 5: 'Sat', 6: 'Sun'} ).astype(str)
train_df['hour'] = train_df['hour'].map({0:'AM1', 1:'AM1', 2:'AM1', 3:'AM2', 4:'AM2', 5:'AM2', 6:'AM3', 7:'AM3', 8:'AM3',
                                         9:'AM4', 10:'AM4', 11:'AM4', 12:'PM1', 13:'PM1', 14:'PM1', 15:'PM2', 16:'PM2',
                                         17:'PM2', 18:'PM3', 19:'PM3', 20:'PM3', 21:'PM4', 22:'PM4', 23:'PM4'}).astype(str)
train_df['day'] = train_df['day'].map({1:'VEAR', 2:'VEAR', 3:'VEAR', 4:'VEAR', 5:'VEAR', 6:'EAR', 7:'EAR', 8:'EAR',
                                       9:'EAR', 10:'EAR', 11:'MID', 12:'MID', 13:'MID', 14:'MID', 15:'MID', 16:'VMID',
                                       17:'VMID', 18:'VMID', 19:'VMID', 20:'VMID', 21:'LAT', 22:'LAT', 23:'LAT', 
                                       24:'LAT', 25:'LAT', 26:'VLAT', 27:'VLAT', 28:'VLAT', 29:'VLAT', 30:'VLAT',
                                       31:'VLAT'}).astype(str)

In [8]:
train_df.head()

,id,user_id,campaign_id,send_date,is_open,is_click,communication_type,total_links,no_of_internal_links,no_of_images,no_of_sections,email_body,subject,email_url
0,42_14051,14051,42,01-09-2017 19:55,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
1,42_177808,177808,42,01-09-2017 20:13,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
2,42_133077,133077,42,01-09-2017 20:11,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
3,42_118677,118677,42,01-09-2017 20:15,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...
4,42_25809,25809,42,01-09-2017 19:49,0,0,Newsletter,88,79,13,4,"September Newsletter\r\n \r\nDear AVians,\r\n ...",[September] Exciting days ahead with DataHack ...,http://r.newsletters.analyticsvidhya.com/7v3rd...


In [5]:
print('Grouping variables creation')
# ip - user_id,  Channel - campaign_id
gp = train_df[['user_id','campaign_id']].groupby(by=['user_id'])[['campaign_id']].count().reset_index().rename(index=str, columns={'campaign_id': 'totcnt'})
train_df = train_df.merge(gp, on=['user_id'], how='left')
del gp
gc.collect()
gp = train_df[['user_id','is_open']].groupby(by=['user_id'])[['is_open']].sum().reset_index().rename(index=str, columns={'is_open': 'opncnt'})
train_df = train_df.merge(gp, on=['user_id'], how='left')
del gp
gc.collect()
gp = train_df[['user_id','is_click']].groupby(by=['user_id'])[['is_click']].sum().reset_index().rename(index=str, columns={'is_click': 'clkcnt'})
train_df = train_df.merge(gp, on=['user_id'], how='left')
del gp
gc.collect()

gp = train_df[['user_id','dow','campaign_id']].groupby(by=['user_id','dow'])[['campaign_id']].count().reset_index().rename(index=str, columns={'campaign_id': 'dowcnt'})
train_df = train_df.merge(gp, on=['user_id','dow'], how='left')
del gp
gc.collect()
gp = train_df[['user_id','dow','is_click']].groupby(by=['user_id','dow'])[['is_click']].sum().reset_index().rename(index=str, columns={'is_click': 'dowclk'})
train_df = train_df.merge(gp, on=['user_id','dow'], how='left')
del gp
gc.collect()

gp = train_df[['user_id','day','campaign_id']].groupby(by=['user_id','day'])[['campaign_id']].count().reset_index().rename(index=str, columns={'campaign_id': 'daycnt'})
train_df = train_df.merge(gp, on=['user_id','day'], how='left')
del gp
gc.collect()
gp = train_df[['user_id','day','is_click']].groupby(by=['user_id','day'])[['is_click']].sum().reset_index().rename(index=str, columns={'is_click': 'dayclk'})
train_df = train_df.merge(gp, on=['user_id','day'], how='left')
del gp
gc.collect()

gp = train_df[['user_id','hour','campaign_id']].groupby(by=['user_id','hour'])[['campaign_id']].count().reset_index().rename(index=str, columns={'campaign_id': 'hourcnt'})
train_df = train_df.merge(gp, on=['user_id','hour'], how='left')
del gp
gc.collect()
gp = train_df[['user_id','hour','is_click']].groupby(by=['user_id','hour'])[['is_click']].sum().reset_index().rename(index=str, columns={'is_click': 'hourclk'})
train_df = train_df.merge(gp, on=['user_id','hour'], how='left')
del gp
gc.collect()

Grouping variables creation
Assigning data types for variables


In [ ]:
print("Assigning data types for variables")
train_df['totcnt'] = train_df['totcnt'].fillna(0).astype('uint16')
train_df['opncnt'] = train_df['opncnt'].fillna(0).astype('uint16')
train_df['clkcnt'] = train_df['clkcnt'].fillna(0).astype('uint16')
train_df['dowcnt'] = train_df['dowcnt'].fillna(0).astype('uint16')
train_df['dowclk'] = train_df['dowclk'].fillna(0).astype('uint16')
train_df['daycnt'] = train_df['daycnt'].fillna(0).astype('uint16')
train_df['dayclk'] = train_df['dayclk'].fillna(0).astype('uint16')
train_df['hourcnt'] = train_df['hourcnt'].fillna(0).astype('uint16')
train_df['hourclk'] = train_df['hourclk'].fillna(0).astype('uint16')

In [6]:
test = train_df[len_train:]
train = train_df[:len_train]
print(train.shape,test.shape)

(1023191, 26) (773858, 26)


In [7]:
train.columns

Index(['campaign_id', 'communication_type', 'email_body', 'email_url', 'id',
       'is_click', 'is_open', 'no_of_images', 'no_of_internal_links',
       'no_of_sections', 'send_date', 'subject', 'total_links', 'user_id',
       'hour', 'day', 'dow', 'totcnt', 'opncnt', 'clkcnt', 'dowcnt', 'dowclk',
       'daycnt', 'dayclk', 'hourcnt', 'hourclk'],
      dtype='object')

In [8]:
# prep data for Base Model
target_train = train['is_click']
id_test = test['id']

del train['campaign_id'],train['id'],train['user_id'],train['is_open'],train['is_click'],train['email_body'],train['subject'],train['email_url'],train['send_date'],train['no_of_images'],train['no_of_internal_links'],train['no_of_sections'],train['total_links']
del test['campaign_id'],test['id'],test['user_id'],test['is_open'],test['is_click'],test['email_body'],test['subject'],test['email_url'],test['send_date'],test['no_of_images'],test['no_of_internal_links'],test['no_of_sections'],test['total_links']

print(train.shape,test.shape)

(1023191, 13) (773858, 13)


In [9]:
train_cats = pd.get_dummies(data=train, columns=['communication_type','dow','hour','day'], drop_first=True)
test_cats = pd.get_dummies(data=test, columns=['communication_type','dow','hour','day'], drop_first=True)
print (train_cats.shape,test_cats.shape)

(1023191, 29) (773858, 27)


In [10]:
train_cats.columns

Index(['totcnt', 'opncnt', 'clkcnt', 'dowcnt', 'dowclk', 'daycnt', 'dayclk',
       'hourcnt', 'hourclk', 'communication_type_Corporate',
       'communication_type_Hackathon', 'communication_type_Newsletter',
       'communication_type_Others', 'communication_type_Upcoming Events',
       'communication_type_Webinar', 'dow_Mon', 'dow_Sat', 'dow_Sun',
       'dow_Thr', 'dow_Tue', 'dow_Wed', 'hour_PM1', 'hour_PM2', 'hour_PM3',
       'hour_PM4', 'day_LAT', 'day_MID', 'day_VLAT', 'day_VMID'],
      dtype='object')

In [11]:
listA = list(train_cats.columns.values)
listB = list(test_cats.columns.values)

In [12]:
for item in listA:
    if item not in listB:
        print (item)

communication_type_Corporate
communication_type_Others
communication_type_Webinar
day_LAT
day_VLAT


In [13]:
for item in listB:
    if item not in listA:
        print (item)

hour_AM3
hour_AM4
day_VEAR


In [14]:
test_cats['communication_type_Corporate'],test_cats['communication_type_Others'],test_cats['communication_type_Webinar'],test_cats['day_LAT'],test_cats['day_VLAT']=0,0,0,0,0
train_cats['hour_AM3'],train_cats['hour_AM4'],train_cats['hour_AM4'] =0,0,0

In [15]:
train_cats.columns

Index(['totcnt', 'opncnt', 'clkcnt', 'dowcnt', 'dowclk', 'daycnt', 'dayclk',
       'hourcnt', 'hourclk', 'communication_type_Corporate',
       'communication_type_Hackathon', 'communication_type_Newsletter',
       'communication_type_Others', 'communication_type_Upcoming Events',
       'communication_type_Webinar', 'dow_Mon', 'dow_Sat', 'dow_Sun',
       'dow_Thr', 'dow_Tue', 'dow_Wed', 'hour_PM1', 'hour_PM2', 'hour_PM3',
       'hour_PM4', 'day_LAT', 'day_MID', 'day_VLAT', 'day_VMID', 'hour_AM3',
       'hour_AM4'],
      dtype='object')

In [16]:
train_cats = train_cats[['totcnt', 'opncnt', 'clkcnt', 'dowcnt', 'dowclk', 'daycnt', 'dayclk','hourcnt', 'hourclk',
                        'communication_type_Corporate','communication_type_Hackathon','communication_type_Newsletter',
                        'communication_type_Others','communication_type_Upcoming Events','communication_type_Webinar',
                        'dow_Mon', 'dow_Sat', 'dow_Sun','dow_Thr', 'dow_Tue', 'dow_Wed', 'hour_PM1', 'hour_PM2',
                        'hour_PM3','hour_PM4', 'day_LAT', 'day_MID', 'day_VLAT', 'day_VMID', 'hour_AM3','hour_AM4']]

test_cats = test_cats[['totcnt', 'opncnt', 'clkcnt', 'dowcnt', 'dowclk', 'daycnt', 'dayclk','hourcnt', 'hourclk',
                      'communication_type_Corporate','communication_type_Hackathon','communication_type_Newsletter',
                      'communication_type_Others','communication_type_Upcoming Events','communication_type_Webinar',
                      'dow_Mon', 'dow_Sat', 'dow_Sun','dow_Thr', 'dow_Tue', 'dow_Wed', 'hour_PM1', 'hour_PM2',
                      'hour_PM3','hour_PM4', 'day_LAT', 'day_MID', 'day_VLAT', 'day_VMID', 'hour_AM3','hour_AM4']]

In [17]:
traindf = np.array(train_cats)
testdf = np.array(test_cats)

xgb_preds = []

In [18]:
K = 5
kf = KFold(n_splits = K, random_state = 42, shuffle = True)

In [21]:
start = time()

for train_index, test_index in kf.split(traindf):
    train_X, valid_X = traindf[train_index], traindf[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
    xgb_params = {'eta': 0.02, 'objective': 'binary:logistic', max_depth= 6, subsample = 1, colsample_bytree = 1,
                  min_chil_weight=1, 'eval_metric': 'auc', 'seed': 42, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(testdf)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, 
                      maximize=True, verbose_eval=50, early_stopping_rounds=50)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

end = time()
print ('Time taken is:', end-start)

SyntaxError: invalid syntax (<ipython-input-21-a6849180cd34>, line 9)

In [70]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

out = pd.DataFrame({'id': id_test, 'is_click': preds})
out.to_csv("pred3.csv", index=False)